
swimmers:

    unique id
    name 
    age 


strokes:

    unique id 
    distance 
    stroke 


times:

    swimmer_id
    stroke_id
    time 
    timestamp


create database swimdataDB;

grant all on swimdataDB.* to 'swimuser'@'localhost' identified by 'swimpasswd';

create table swimmers (
    id int not null auto_increment primary key,
    name varchar(32) not null,
    age int not null
);

create table strokes (
    id int not null auto_increment primary key,
    distance varchar(16) not null,
    stroke varchar(16) not null
);

create table times (
    swimmer_id int not null,
    stroke_id int not null,
    time varchar(16) not null,
    ts timestamp default current_timestamp
);

----------------------------------------------------------------------

MariaDB [swimdataDB]> describe strokes;
+----------+-------------+------+-----+---------+----------------+
| Field    | Type        | Null | Key | Default | Extra          |
+----------+-------------+------+-----+---------+----------------+
| id       | int(11)     | NO   | PRI | NULL    | auto_increment |
| distance | varchar(16) | NO   |     | NULL    |                |
| stroke   | varchar(16) | NO   |     | NULL    |                |
+----------+-------------+------+-----+---------+----------------+


MariaDB [swimdataDB]> describe swimmers;
+-------+-------------+------+-----+---------+----------------+
| Field | Type        | Null | Key | Default | Extra          |
+-------+-------------+------+-----+---------+----------------+
| id    | int(11)     | NO   | PRI | NULL    | auto_increment |
| name  | varchar(32) | NO   |     | NULL    |                |
| age   | int(11)     | NO   |     | NULL    |                |
+-------+-------------+------+-----+---------+----------------+


MariaDB [swimdataDB]> describe times;
+------------+-------------+------+-----+---------------------+-------+
| Field      | Type        | Null | Key | Default             | Extra |
+------------+-------------+------+-----+---------------------+-------+
| swimmer_id | int(11)     | NO   |     | NULL                |       |
| stroke_id  | int(11)     | NO   |     | NULL                |       |
| time       | varchar(16) | NO   |     | NULL                |       |
| ts         | timestamp   | YES  |     | current_timestamp() |       |
+------------+-------------+------+-----+---------------------+-------+


In [1]:
%pip install mycli --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

FOLDER = "swimdata/"

files = os.listdir(FOLDER)

files.remove(".DS_Store")

In [3]:
len(files)

37

In [4]:
print(files)

['Abi-10-100m-Back.txt', 'Abi-10-100m-Breast.txt', 'Ali-12-100m-Back.txt', 'Ali-12-100m-Free.txt', 'Aurora-13-50m-Free.txt', 'Bill-18-100m-Back.txt', 'Bill-18-200m-Back.txt', 'Calvin-9-50m-Back.txt', 'Calvin-9-50m-Fly.txt', 'Carl-15-100m-Back.txt', 'Chris-17-100m-Back.txt', 'Chris-17-100m-Breast.txt', 'Darius-13-100m-Fly.txt', 'Darius-13-200m-IM.txt', 'Dave-17-100m-Free.txt', 'Dave-17-200m-Back.txt', 'Elba-14-100m-Free.txt', 'Emma-13-100m-Breast.txt', 'Emma-13-100m-Free.txt', 'Lizzie-14-100m-Back.txt', 'Lizzie-14-100m-Free.txt', 'Maria-9-50m-Free.txt', 'Mike-15-100m-Back.txt', 'Mike-15-100m-Fly.txt', 'Mike-15-100m-Free.txt', 'Mike-15-200m-Free.txt', 'Mike-15-200m-IM.txt', 'Owen-15-100m-Free.txt', 'Ruth-13-100m-Back.txt', 'Ruth-13-100m-Free.txt', 'Ruth-13-200m-Back.txt', 'Ruth-13-200m-Free.txt', 'Ruth-13-400m-Free.txt', 'Tasmin-15-100m-Back.txt', 'Tasmin-15-100m-Breast.txt', 'Tasmin-15-100m-Free.txt', 'Tasmin-15-200m-Breast.txt']


In [5]:
import DBcm

In [6]:
config = {
    "user": "swimuser",
    "password": "swimpasswd",
    "host": "localhost",
    "database": "swimdataDB",
}

In [7]:
with DBcm.UseDatabase(config) as db:
    name = "Abi"
    age = 9
    SQL = "select * from swimmers where name = %s and age = %s;"  # %s is a placeholder.
    db.execute(SQL, (name, age))
    results = db.fetchall()

In [8]:
results

[]

In [9]:
def insert_if_not_already_there(connection, table, field1, field2, value1, value2):
    SQL = f"select * from {table} where {field1} = %s and {field2} = %s;"
    connection.execute(SQL, (value1, value2))
    results = connection.fetchall()
    if results:
        pass
    else:
        SQL = f"insert into {table} ({field1}, {field2}) values (%s, %s)"
        db.execute(SQL, (value1, value2))

In [10]:
with DBcm.UseDatabase(config) as db:
    for fn in files:
        name, age, distance, stroke = fn.removesuffix(".txt").split("-")
        insert_if_not_already_there(db, "swimmers", "name", "age", name, age)
        insert_if_not_already_there(
            db, "strokes", "distance", "stroke", distance, stroke
        )

In [11]:
with DBcm.UseDatabase(config) as db:
    SQL = "select id from swimmers where name = 'Darius' and age = 13"
    db.execute(SQL)
    results = db.fetchone()[0]
results

8

In [12]:
def get_id(connection, table, field1, field2, value1, value2):
    SQL = f"select id from {table} where {field1} = %s and {field2} = %s"
    connection.execute(SQL, (value1, value2))
    return connection.fetchone()[0]

In [13]:
import swimclub

with DBcm.UseDatabase(config) as db:
    for fn in files:
        *_, times, _ = swimclub.get_swim_data(fn)
        name, age, distance, stroke = fn.removesuffix(".txt").split("-")
        swimmer_id = get_id(db, "swimmers", "name", "age", name, age)
        stroke_id = get_id(db, "strokes", "distance", "stroke", distance, stroke)
        for t in times:
            SQL = "insert into times (swimmer_id, stroke_id, time) values (%s, %s, %s)"
            db.execute(SQL, (swimmer_id, stroke_id, t))

##### At this point, the data is in the database tables, so we can work with it (as opposed to going back to the filesystem).

In [14]:
import DBcm

config = {
    "user": "swimuser",
    "password": "swimpasswd",
    "host": "localhost",
    "database": "swimdataDB",
}

swimmer_name = "Katie"
swimmer_age = 9

event_distance = "100m"
event_stroke = "Back"

SQL = """
        select swimmers.name, swimmers.age, times.time, strokes.distance, strokes.stroke, times.ts
        from swimmers, times, strokes
        where (swimmers.name = %s and swimmers.age = %s) and
        (strokes.distance = %s and strokes.stroke = %s) and
        swimmers.id = times.swimmer_id and
        strokes.id = times.stroke_id
"""

with DBcm.UseDatabase(config) as db:
    db.execute(
        SQL,
        (
            swimmer_name,
            swimmer_age,
            event_distance,
            event_stroke,
        ),
    )
    results = db.fetchall()

In [15]:
for row in results:
    print(row)

In [16]:
import DBcm

config = {
    "user": "swimuser",
    "password": "swimpasswd",
    "host": "localhost",
    "database": "swimdataDB",
}


def get_swimmers_data(name, age, distance, stroke):
    SQL = """
        select swimmers.name, swimmers.age, times.time, strokes.distance, strokes.stroke, times.ts
        from swimmers, times, strokes
        where (swimmers.name = %s and swimmers.age = %s) and
        (strokes.distance = %s and strokes.stroke = %s) and
        swimmers.id = times.swimmer_id and
        strokes.id = times.stroke_id
    """
    with DBcm.UseDatabase(config) as db:
        db.execute(
            SQL,
            (
                name,
                age,
                distance,
                stroke,
            ),
        )
        results = db.fetchall()
    return results

In [17]:
for row in get_swimmers_data("Darius", 13, "200m", "IM"):
    print(row)

('Darius', 13, '3:03.84', '200m', 'IM', datetime.datetime(2023, 1, 28, 18, 48, 19))
('Darius', 13, '3:04.27', '200m', 'IM', datetime.datetime(2023, 1, 28, 18, 48, 19))
('Darius', 13, '3:01.49', '200m', 'IM', datetime.datetime(2023, 1, 28, 18, 48, 19))
('Darius', 13, '3:04.27', '200m', 'IM', datetime.datetime(2023, 1, 28, 18, 48, 19))
('Darius', 13, '2:52.44', '200m', 'IM', datetime.datetime(2023, 1, 28, 18, 48, 19))


In [18]:
for row in get_swimmers_data("Chris", 17, "100m", "Back"):
    print(row)

('Chris', 17, '1:21.76', '100m', 'Back', datetime.datetime(2023, 1, 28, 18, 48, 19))
('Chris', 17, '1:23.32', '100m', 'Back', datetime.datetime(2023, 1, 28, 18, 48, 19))
('Chris', 17, '1:24.09', '100m', 'Back', datetime.datetime(2023, 1, 28, 18, 48, 19))
('Chris', 17, '1:21.51', '100m', 'Back', datetime.datetime(2023, 1, 28, 18, 48, 19))


In [19]:
# List a named swimmer's events (as stored in the database).

import DBcm

config = {
    "user": "swimuser",
    "password": "swimpasswd",
    "host": "localhost",
    "database": "swimdataDB",
}

swimmer = "Hannah"

SQL = """ 
    select distinct strokes.distance, strokes.stroke
    from swimmers, strokes, times
    where times.swimmer_id = swimmers.id and
    times.stroke_id = strokes.id and
    swimmers.name = %s;
"""

with DBcm.UseDatabase(config) as db:
    db.execute(SQL, (swimmer,))
    results = db.fetchall()
results
## list(set(results))

[]

In [20]:
events = [t[0] + "-" + t[1] for t in results]
events

[]

In [21]:
SQL = "select name from swimmers"
with DBcm.UseDatabase(config) as db:
    db.execute(SQL)
    results = db.fetchall()

In [22]:
results

[('Abi',),
 ('Ali',),
 ('Aurora',),
 ('Bill',),
 ('Calvin',),
 ('Carl',),
 ('Chris',),
 ('Darius',),
 ('Dave',),
 ('Elba',),
 ('Emma',),
 ('Lizzie',),
 ('Maria',),
 ('Mike',),
 ('Owen',),
 ('Ruth',),
 ('Tasmin',)]

In [23]:
names = [t[0] for t in results]  # 't' is the current tuple.

In [24]:
print(sorted(names))

['Abi', 'Ali', 'Aurora', 'Bill', 'Calvin', 'Carl', 'Chris', 'Darius', 'Dave', 'Elba', 'Emma', 'Lizzie', 'Maria', 'Mike', 'Owen', 'Ruth', 'Tasmin']


In [25]:
import data_utils

In [26]:
data_utils.get_list_of_sessions()

[(datetime.datetime(2023, 1, 28, 18, 48, 19),)]

In [27]:
sessions = data_utils.get_list_of_sessions()

In [28]:
sessions

[(datetime.datetime(2023, 1, 28, 18, 48, 19),)]

In [29]:
sessions[0][0]

datetime.datetime(2023, 1, 28, 18, 48, 19)

In [30]:
print(dir(sessions[0][0]))

['__add__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__rsub__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', 'astimezone', 'combine', 'ctime', 'date', 'day', 'dst', 'fold', 'fromisocalendar', 'fromisoformat', 'fromordinal', 'fromtimestamp', 'hour', 'isocalendar', 'isoformat', 'isoweekday', 'max', 'microsecond', 'min', 'minute', 'month', 'now', 'replace', 'resolution', 'second', 'strftime', 'strptime', 'time', 'timestamp', 'timetuple', 'timetz', 'today', 'toordinal', 'tzinfo', 'tzname', 'utcfromtimestamp', 'utcnow', 'utcoffset', 'utctimetuple', 'weekday', 'year']


In [31]:
sessions[0][0].ctime()

'Sat Jan 28 18:48:19 2023'

In [32]:
sessions[0][0].isoformat()

'2023-01-28T18:48:19'

In [33]:
sessions[0][0].isoformat().split("T")[0]

'2023-01-28'

In [34]:
sessions[0][0].isoformat().split("T")[1]

'18:48:19'

In [35]:
for row in sessions:
    print(row[0].isoformat().split("T")[0])

2023-01-28


In [36]:
SQL = """select * from times where date_format(ts, "%Y-%m-%d") = "2022-12-08";"""

In [37]:
sessions = [
    row[0].isoformat().split("T")[0] for row in data_utils.get_list_of_sessions()
]

In [38]:
sessions

['2023-01-28']

In [39]:
sorted(sessions, reverse=True)

['2023-01-28']

In [40]:
sorted(["2022-12-08", "2022-12-01", "2026-01-01"], reverse=True)

['2026-01-01', '2022-12-08', '2022-12-01']

In [41]:
SQL = """
    select distinct swimmers.name   
    from times, swimmers 
    where date_format(times.ts, "%Y-%m-%d") = %s and     
    times.swimmer_id = swimmers.id 
    order by name
"""

In [42]:
with DBcm.UseDatabase(config) as db:
    db.execute(SQL, (sessions[0],))
    results = db.fetchall()
[row[0] for row in results]

['Abi',
 'Ali',
 'Aurora',
 'Bill',
 'Calvin',
 'Carl',
 'Chris',
 'Darius',
 'Dave',
 'Elba',
 'Emma',
 'Lizzie',
 'Maria',
 'Mike',
 'Owen',
 'Ruth',
 'Tasmin']